In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
from sklearn.preprocessing import OneHotEncoder
from torch.utils.data import Dataset, DataLoader
import numpy as np
from auxiliary import ScaledEmbedding, ZeroEmbedding
import data_loader
import itertools

class PT(nn.Module):
    def get_params(self, users, items):
        return self.al
    def __init__(self, userLen, itemLen, distribution, params, item_price):
        super(PT, self).__init__()
        self.userNum = userLen
        self.itemNum = itemLen
        self.params = params

        if 'gpu' in params and params['gpu'] == True:
            self.device = 'cuda'
        else:
            self.device = 'cpu'

        l_size = params['l_size']
        self.distribution = torch.FloatTensor(distribution).to(self.device)
        self.item_price = torch.FloatTensor(item_price).to(self.device)
        self.globalBias_g = ZeroEmbedding(1, 1).to(self.device).to(torch.float)
        self.globalBias_g.weight.data += 0.5
        self.globalBias_g.weight.requires_grad = False
        self.ecoBias_g = ZeroEmbedding(itemLen, 1).to(self.device).to(torch.float)
        self.userBias_g = ZeroEmbedding(userLen, 1).to(self.device).to(torch.float)
        self.itemBias_g = ZeroEmbedding(itemLen, 1).to(self.device).to(torch.float)
        self.userEmbed_g = ScaledEmbedding(userLen, l_size).to(self.device).to(torch.float)
        self.ecoEmbed_g = ScaledEmbedding(itemLen, l_size).to(self.device).to(torch.float)
        self.itemEmbed_g = ScaledEmbedding(itemLen, l_size).to(self.device).to(torch.float)

        self.globalBias_d = ZeroEmbedding(1, 1).to(self.device).to(torch.float)
        self.globalBias_d.weight.data += 0.5
        self.globalBias_d.weight.requires_grad = False
        self.ecoBias_d = ZeroEmbedding(itemLen, 1).to(self.device).to(torch.float)
        self.userBias_d = ZeroEmbedding(userLen, 1).to(self.device).to(torch.float)
        self.itemBias_d = ZeroEmbedding(itemLen, 1).to(self.device).to(torch.float)
        self.userEmbed_d = ScaledEmbedding(userLen, l_size).to(self.device).to(torch.float)
        self.ecoEmbed_d = ScaledEmbedding(itemLen, l_size).to(self.device).to(torch.float)
        self.itemEmbed_d = ScaledEmbedding(itemLen, l_size).to(self.device).to(torch.float)

        self.globalBias_a = ZeroEmbedding(1, 1).to(self.device).to(torch.float)
        self.globalBias_a.weight.requires_grad = False
        self.userBias_a = ZeroEmbedding(userLen, 1).to(self.device).to(torch.float)
        self.userBias_a.weight.data.uniform_(0.0, 0.05)
        self.itemBias_a = ZeroEmbedding(itemLen, 1).to(self.device).to(torch.float)
        self.ecoBias_a = ZeroEmbedding(itemLen, 1).to(self.device).to(torch.float)
        self.itemBias_a.weight.data.uniform_(0.0, 0.05)
        self.userEmbed_a = ZeroEmbedding(userLen, l_size).to(self.device).to(torch.float)
        self.userEmbed_a.weight.data.uniform_(-0.01, 0.01)
        self.itemEmbed_a = ZeroEmbedding(itemLen, l_size).to(self.device).to(torch.float)
        self.itemEmbed_a.weight.data.uniform_(-0.01, 0.01)
        self.ecoEmbed_a = ZeroEmbedding(itemLen, l_size).to(self.device).to(torch.float)
        self.ecoEmbed_a.weight.data.uniform_(-0.01, 0.01)

        self.globalBias_b = ZeroEmbedding(1, 1).to(self.device).to(torch.float)
        self.globalBias_b.weight.requires_grad = False
        self.userBias_b = ZeroEmbedding(userLen, 1).to(self.device).to(torch.float)
        self.ecoBias_b = ZeroEmbedding(itemLen, 1).to(self.device).to(torch.float)
        self.userBias_b.weight.data.uniform_(0.0, 0.05)
        self.ecoBias_b.weight.data.uniform_(0.0, 0.05)
        self.itemBias_b = ZeroEmbedding(itemLen, 1).to(self.device).to(torch.float)
        self.itemBias_b.weight.data.uniform_(0.0, 0.05)
        self.userEmbed_b = ZeroEmbedding(userLen, l_size).to(self.device).to(torch.float)
        self.userEmbed_b.weight.data.uniform_(-0.01, 0.01)
        self.itemEmbed_b = ZeroEmbedding(itemLen, l_size).to(self.device).to(torch.float)
        self.itemEmbed_b.weight.data.uniform_(-0.01, 0.01)
        self.ecoEmbed_b = ZeroEmbedding(itemLen, l_size).to(self.device).to(torch.float)
        self.itemEmbed_b.weight.data.uniform_(-0.01, 0.01)

        self.ecoEmbed_l = ZeroEmbedding(itemLen, l_size).to(self.device).to(torch.float)
        self.globalBias_l = ZeroEmbedding(1, 1).to(self.device).to(torch.float)
        self.globalBias_l.weight.data += 1
        self.globalBias_l.weight.requires_grad = False
        self.userBias_l = ZeroEmbedding(userLen, 1).to(self.device).to(torch.float)
        self.userBias_l.weight.data.uniform_(0.0, 0.05)
        self.itemBias_l = ZeroEmbedding(itemLen, 1).to(self.device).to(torch.float)
        self.itemBias_l.weight.data.uniform_(0.0, 0.05)
        self.ecoBias_l = ZeroEmbedding(itemLen, 1).to(self.device).to(torch.float)
        self.ecoBias_l.weight.data.uniform_(0.0, 0.05)
        self.userEmbed_l = ZeroEmbedding(userLen, l_size).to(self.device).to(torch.float)
        self.userEmbed_l.weight.data.uniform_(-0.01, 0.01)
        self.itemEmbed_l = ZeroEmbedding(itemLen, l_size).to(self.device).to(torch.float)
        self.itemEmbed_l.weight.data.uniform_(-0.01, 0.01)
        self.ecoEmbed_l = ZeroEmbedding(itemLen, l_size).to(self.device).to(torch.float)
        self.ecoEmbed_l.weight.data.uniform_(-0.01, 0.01)

        self.reference_point = ZeroEmbedding(userLen, 1).to(self.device).to(torch.float)
        self.reference_point.weight.data = torch.ones_like(self.reference_point.weight.data) * 1.5
        #		 self.reference_point.weight.requires_grad=False
        self.to(self.device)
        self.grads = {}
    def ecoForward(self, items):
        ecoBias_a = self.ecoBias_a(items)
        ecoEmbed_a = self.ecoEmbed_a(items)
        itemEmbed_a = self.itemEmbed_a(items)

        alpha = ecoBias_a + torch.mul(ecoEmbed_a, itemEmbed_a).sum(1).view(-1, 1)
        return alpha

    def forward(self, users, items):
        distribution = self.distribution[items].to(self.device)
        reference_point = self.reference_point(users)
        #		 print(users.shape[0],items.shape[0])
        price = self.item_price[items].view(-1, 1).expand(users.shape[0], 5).to(self.device)

        # calculate value
        globalBias_a = self.globalBias_a(torch.tensor(0).to(self.device))
        userBias_a = self.userBias_a(users)
        itemBias_a = self.itemBias_a(items)
        userEmbed_a = self.userEmbed_a(users)
        itemEmbed_a = self.itemEmbed_a(items)

        globalBias_b = self.globalBias_b(torch.tensor(0).to(self.device))
        userBias_b = self.userBias_b(users)
        itemBias_b = self.itemBias_b(items)
        userEmbed_b = self.userEmbed_b(users)
        itemEmbed_b = self.itemEmbed_b(items)

        globalBias_l = self.globalBias_l(torch.tensor(0).to(self.device))
        userBias_l = self.userBias_l(users)
        itemBias_l = self.itemBias_l(items)
        userEmbed_l = self.userEmbed_l(users)
        itemEmbed_l = self.itemEmbed_l(items)

        alpha = globalBias_a + userBias_a + itemBias_a + torch.mul(userEmbed_a, itemEmbed_a).sum(1).view(-1, 1)
        beta = globalBias_b + userBias_b + itemBias_b + torch.mul(userEmbed_b, itemEmbed_b).sum(1).view(-1, 1)
        lamda = globalBias_l + userBias_l + itemBias_l + torch.mul(userEmbed_l, itemEmbed_l).sum(1).view(-1, 1)

        rating = torch.tensor([1., 2., 3., 4., 5.]).expand(users.shape[0], 5).to(self.device)
        x = torch.tanh(rating - reference_point)
        x_binary_pos = torch.gt(x, torch.FloatTensor([0]).to(self.device)).to(torch.float)
        x_binary_neg = torch.ones_like(x).to(self.device) - x_binary_pos

        x_ = torch.mul(price, torch.abs(x))
        v_exp = torch.mul(alpha, x_binary_pos) + torch.mul(beta, x_binary_neg)
        v = x_.pow(v_exp)
        v_coef = x_binary_pos - torch.mul(lamda, x_binary_neg)
        value = torch.mul(v, v_coef).to(self.device)

        # calculate weight
        globalBias_g = self.globalBias_g(torch.tensor(0).to(self.device))
        userBias_g = self.userBias_g(users)
        itemBias_g = self.itemBias_g(items)
        userEmbed_g = self.userEmbed_g(users)
        itemEmbed_g = self.itemEmbed_g(items)

        globalBias_d = self.globalBias_d(torch.tensor(0).to(self.device))
        userBias_d = self.userBias_d(users)
        itemBias_d = self.itemBias_d(items)
        userEmbed_d = self.userEmbed_d(users)
        itemEmbed_d = self.itemEmbed_d(items)

        gamma = globalBias_g + userBias_g + itemBias_g + torch.mul(userEmbed_g, itemEmbed_g).sum(1).view(-1, 1)
        delta = globalBias_d + userBias_d + itemBias_d + torch.mul(userEmbed_d, itemEmbed_d).sum(1).view(-1, 1)

        gamma_ = gamma.expand(users.shape[0], 5)
        delta_ = delta.expand(users.shape[0], 5)
        w_exp = torch.mul(x_binary_pos, gamma_) + torch.mul(x_binary_neg, delta_)

        w_nominator = distribution.pow(w_exp)
        w_denominator = (distribution.pow(w_exp) + (torch.ones_like(distribution).to(self.device) - distribution).pow(
            w_exp)).pow(1 / w_exp)
        weight = torch.div(w_nominator, w_denominator)

        #		 self.userBias_g.weight.register_hook(self.save_grad('userBias_g'))
        #		 self.itemBias_g.weight.register_hook(self.save_grad('itemBias_g'))
        #		 self.userEmbed_g.weight.register_hook(self.save_grad('userEmbed_g'))
        #		 self.itemEmbed_g.weight.register_hook(self.save_grad('itemEmbed_g'))
        return torch.mul(weight, value).sum(1)

    def getValues(self, users, items):
        pOut = float(self.forward(users, items))  # .expand(users.shape[0], self.params['negNum_train']).reshape(-1, 1)
        return pOut

    def get_grads(self):
        return self.grads

    def save_grad(self, name):
        def hook(grad):
            self.grads[name] = grad

        return hook

if __name__ == '__main__':
    params = dict()
    params['lr'] = 1e-4
    params['batch_size'] = 1
    params['epoch_limit'] = 1000
    params['epoch_diff'] = 999
    params['w_decay'] = 5e-4
    params['negNum_test'] = 36
    params['epsilon'] = 1e-4
    params['negNum_train'] = 2
    params['l_size'] = 128
    params['train_device'] = 'cpu'
    params['test_device'] = 'cpu'
    params['lambda'] = 1.
    params['test_per_train'] = 1
    item_price = np.load(r"C:\\Users\march\Risk-Aware-Recommnedation-Model\data\Movielens1M_item_price.npy")
    category1 = 'newTrainSamples'
    category2 = 'newTestSamples'
    catAll = 'AllSamples'
    metaCat = 'Appliances'

    train, test = data_loader.read_data(category1, category2)
    userNum, itemNum = data_loader.get_datasize(catAll)
    data_loader.get_ecoScores('Appliances', catAll)
    AllSamples = data_loader.read_AllSamples(catAll)
    frequency = data_loader.get_itemDist(AllSamples, itemNum)
    distribution = data_loader.approx_Gaussian(frequency)

    trainset = data_loader.TransactionData(train, userNum, itemNum, frequency)
    trainLoader = DataLoader(trainset, batch_size=params['batch_size'], shuffle=False, num_workers=0)

    trainset = data_loader.TransactionData(train, userNum, itemNum, frequency)
    trainLoader = DataLoader(trainset, batch_size=params['batch_size'], shuffle=False, num_workers=0)

    model = PT(userLen=userNum, itemLen=itemNum, distribution=distribution, params=params, item_price=item_price)

    knn = KNeighborsClassifier(n_neighbors=5, metric='minkowski', p=2)

    item_price = np.load(r"C:\\Users\march\Risk-Aware-Recommnedation-Model\data\Movielens1M_item_price.npy")
    category1 = 'newTrainSamples'
    category2 = 'newTestSamples'
    catAll = 'AllSamples'
    metaCat = 'Appliances'

    train, test = data_loader.read_data(category1, category2)
    userNum, itemNum = data_loader.get_datasize(catAll)
    data_loader.get_ecoScores(metaCat, catAll)
    AllSamples = data_loader.read_AllSamples(catAll)
    frequency = data_loader.get_itemDist(AllSamples, itemNum)
    distribution = data_loader.approx_Gaussian(frequency)
    meta = data_loader.ecoList

    trainset = data_loader.TransactionData(train, userNum, itemNum, frequency)
    trainLoader = DataLoader(trainset, batch_size=params['batch_size'], shuffle=False, num_workers=0)

    trainset = data_loader.TransactionData(train, userNum, itemNum, frequency)
    trainLoader = DataLoader(trainset, batch_size=params['batch_size'], shuffle=False, num_workers=0)

    model = PT(userLen=userNum, itemLen=itemNum, distribution=distribution, params=params, item_price=item_price)

    model.device = params['train_device']
    model.to(model.device)

    values = []
    L = len(trainLoader.dataset)
    for i, batchData in enumerate(trainLoader):
        users = torch.LongTensor(batchData['user']).to(model.device)
        items = torch.LongTensor(batchData['item']).to(model.device)
        values.append(model.getValues(users,items))
    xTrain = []
    xTest = []
    yTrain = []
    ecoYTrain = []
    yTest = []
    ecoYTest = []

    idx = 0
    for x in range(len(train)):
        xTrain.append([x for x in distribution[train[x][1]]])
        xTrain[idx].append(values[idx])
        xTrain[idx].append([x for x in OneHotEncoder().fit_transform([meta[train[x][1]][0].split()]).data])
        yTrain.append(train[x][2])
        for k in data_loader.items:
            if data_loader.items[k][1] == train[x][1]:
                ecoYTrain.append(data_loader.items[k][0])
        idx += 1
    idx = 0
    for x in range(len(test)):
        xTest.append([x for x in distribution[test[x][1]]])
        xTest[idx].append(values[idx])
        xTest[idx].append([x for x in OneHotEncoder().fit_transform([meta[test[x][1]][0].split()]).data])
        yTest.append(test[x][2])
        for k in data_loader.items:
            if data_loader.items[k][1] == test[x][1]:
                ecoYTest.append(data_loader.items[k][0])
        idx += 1
    flat = []
    idx = 0
    max = len(xTrain[0])
    for x in xTrain:
        for j in x:
            if type(j) == list:
                for k in j:
                    flat.append(k)
            else:
                flat.append(j)
        xTrain[idx] = flat.copy()
        if len(xTrain[idx]) > max:
            max = len(xTrain[idx])
        flat = []
        idx += 1
    for item in range(len(xTrain)):
        if len(xTrain[item]) < max:
            for k in range(max - len(xTrain[item])):
                xTrain[item].append(0.0)
    idx = 0
    for x in xTest:
        for j in x:
            if type(j) == list:
                for k in j:
                    flat.append(k)
            else:
                flat.append(j)
        xTest[idx] = flat.copy()
        flat = []
        idx += 1
    for item in range(len(xTest)):
        if len(xTest[item]) < max:
            for k in range(max - len(xTest[item])):
                xTest[item].append(0.0)
    knn = KNeighborsClassifier(n_neighbors=5, metric= 'minkowski', p=2)
    print("Utility epoch limit: " + str(params['epoch_limit']))
    for i in range(params['epoch_limit']):
        knn.fit(xTrain, yTrain)
    #print(knn.kneighbors)
    for j in range(params['epoch_limit']-params['epoch_diff']):
        knn.fit(xTrain, ecoYTrain)
    print("Utility: ")
    print(knn.score(xTest, yTest))
    print("Eco epoch limit: " + str(params['epoch_limit']-params['epoch_diff']))
    print("Eco-friendliness: ")
    print(knn.score(xTest, ecoYTest))
